<a href="https://colab.research.google.com/github/netaamz/python-quiz-google-colab/blob/main/pythongame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

installations



In [1]:
!pip install firebase
!pip install Jinja2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Login screen**

> player
1. user name: ron
2. password 123456




> manager
1.   user name: sarah
2.   password 123456



In [10]:
from pickle import APPEND
#@title Default title text
from IPython.display import display, HTML, clear_output
import google.colab.output
from firebase import firebase
import random
##imports and drive connect
import IPython
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/pygame/
from jinja2 import Template
import time

%store -r manager_code
%store -r html_mainScreen



FBconn = firebase.FirebaseApplication('https://pythongame-a4fc4-default-rtdb.firebaseio.com/', None)
numOfQuestion = 0
user = None
score=0
html_loginFailed = '''
        <h1 id="login-failed">Login Failed. Please try again.</h1>
        '''
html_login = '''
<!DOCTYPE html>
<html>
<div class="backgrounLogin">
<head>
    <title>Login</title>
    <link rel="stylesheet" type="text/css" href="https://drive.google.com/uc?id=1WQCIPgq9rt5VkLC2H89zB_-esW7iaLGT">
    <img src="https://www.happiness.co.il/media/catalog/product/cache/c03ce33b703f4dbb560edfd071511fcd/2/_/2_9969317.jpg" class=upper-left>
    <script>

        function login() {
            var username = document.getElementById('username').value;
            var password = document.getElementById('password').value;
            // Send the username and password to the Python backend
            google.colab.kernel.invokeFunction('handle_login', [username, password], {});
        }
    </script>
</head>
</div>
<body>
    <div id="login-screen">
        <div class="login-container">
            <h1>Login</h1>
            <label for="username">Username:</label>
            <input type="text" id="username" name="username" placeholder="Enter your username">
            <label for="password">Password:</label>
            <input type="password" id="password" name="password" placeholder="Enter your password">
            <input type="button" class="login-btn" value="Login" onclick="login()">
        </div>
    </div>
</body>
</html>

'''


class User:
    def __init__(self, username, password, id, role):
        self.id = id
        self.username = username
        self.password = password
        self.correct_answers = [False] * 10
        self.role=role

    def set_answer(self, index, value):
        if 0 <= index < len(self.correct_answers):
            self.correct_answers[index] = value
            print(self.correct_answers)

    def get_answer(self, index):
        if 0 <= index < len(self.correct_answers):
            return self.correct_answers[index]
        return None
    def user_details(self):
      return self.id + "," + "," +self.username + ", " + self.password

    def get_username(self):
      return self.username


def isUserExist(username, password):
    global user
    users = FBconn.get('/users/', None)
    for user_id, user_data in users.items():
        if user_data.get('username') == username and user_data.get('password') == password:
            role = user_data.get('role')
            user = User(username,password,user_id,role)
            return True
    return False

def login(username, password):
    if isUserExist(username, password):
        if user.role == "manager":
            clear_output()
            display(HTML(manager_code))
        else:
          clear_output()
          display(HTML(html_mainScreen))  # Display the quiz screen
          display(HTML('''
          <script>
              document.getElementById('login-screen').style.display = 'none';
              document.querySelector('.popup-container').style.display = 'none';
          </script>
          '''))
    else:
        html_code = '''
        <div class="errorContainer"><label id="login-failed">Login Failed. Please try again.</label></div>
        '''
        display(HTML(html_code))


google.colab.output.register_callback('handle_login', login)
clear_output()

display(HTML(html_login))




Question Number,Answered
What is the keyword used to define a function in Python?,🗸
What is the result of 2 + 3 * 4?,🗸
What is the correct syntax for a for loop in Python?,🗸
Which of the following is a data type in Python?,🗸
Which of the following is NOT a Python built-in data type?,🗸
Which Python framework is commonly used for web development?,🗸
Which cloud computing platform is offered by IBM?,🗸
"In Google Colab, True or False: You can execute shell commands directly within a code cell.",🗸
Which of the following data types in Python is used to store textual data?,🗸
Which company offers the AWS (Amazon Web Services) cloud platform?,🗸


Main screen

In [3]:
html_mainScreen = '''
<!DOCTYPE html>
<html>
<head>
  <title>Main Menu</title>
 <link rel="stylesheet" type="text/css" href="https://drive.google.com/uc?id=1WQCIPgq9rt5VkLC2H89zB_-esW7iaLGT">
  <script>
    function logout() {
      google.colab.kernel.invokeFunction('handle_logout');
    }

    function showPopup() {
      var popupContainer = document.getElementById('popup-container');
      popupContainer.style.display = 'block';
    }

    function closePopup() {
      var popupContainer = document.getElementById('popup-container');
      popupContainer.style.display = 'none';
    }
    function openQuestions(){
      google.colab.kernel.invokeFunction('handle_quiz');
    }
  </script>
</head>
<body>
<div id="menu" class="container" style="width: 40rem;">
    <h1>Python Quiz</h1>
    <div class="menu">
      <button class="button help-button" onclick="showPopup()">?</button>
      <button class="button" onclick="openQuestions()">Play</button>
      <button class="button">My Score</button>
      <button class="button">Ranking</button>
    </div>
    <button class="button logout-button" onclick="logout()">Logout</button>
</div>

  <div id="popup-container" class="popup-container">
    <div class="popup">
      <h2>Help</h2>
      <p>Hi, welcome to pythonGame, the trivia game that will teach you Python in the best way! there are questions according to levels, all you need to do is to answer the right question and you are ready! So Lets go we waiting for you!.</p>
      <button class="button close-button" onclick="closePopup()">Close</button>
    </div>
  </div>
</body>
</html>
'''
%store html_mainScreen

def logout():
    clear_output()
    global user
    user=None
    display(HTML(html_login))  # Display the login screen

def openQuiz():
    global numOfQuestion
    global score
    numOfQuestion=0
    score=0
    if(numOfQuestion < len(questions)-1):
        numOfQuestion += 1
        clear_output()
        displayQuestion(numOfQuestion, questions[numOfQuestion], score)

google.colab.output.register_callback('handle_logout', logout)
google.colab.output.register_callback('handle_quiz', openQuiz)

Stored 'html_mainScreen' (str)


Game

In [4]:
import random
from IPython.display import clear_output, display, HTML
#%store -r html_end_game

html_questions = '''
<!DOCTYPE html>
<html>
<head>
    <title>Quiz Question</title>
 <link rel="stylesheet" type="text/css" href="https://drive.google.com/uc?id=1WQCIPgq9rt5VkLC2H89zB_-esW7iaLGT">

     <style>

        .progress-bar {
            width: 80%;
            background-color: #f2f2f2;
            margin: 5px;
            border-radius: 10px;

        }

        .progress-bar-fill {
            height: 20px;
            background-color: #4caf50;
            width: {{ progress }}%;
            transition: width 0.3s ease;
            border-radius: 10px;
        }
    </style>
</head>
<body>

  <div class ="game-quiz-container">
  <div   class="exit">
   <button class="btn-close" onclick="endGame()">&#x274C</button>
   </div>
    <div class="game-details-container ">
        <h1>Score : <span id="player-score">{{ score }}</span> / 10</h1>
        <h1>Question<span id="question_number"> {{ question_num }}</span></h1>
    </div>
     <div class="progress-bar">
            <div class="progress-bar-fill" style="width: {{ progress }}%"></div>
        </div>
    <div class="game-question-container">
        <p>{{ question }}</p>
    </div>

    <div class="game-options-container">
      <span>
        <input type="radio" class="radio" name="answer" id="option1" value="option1">
        <label for="option1" id="option1">{{ option1 }}</label>
      </span>
      <span>
        <input type="radio" class="radio" name="answer" id="option2" value="option2">
        <label for="option2" id="option2">{{ option2 }}</label>
      </span>
      <span>
       <input type="radio" class="radio" name="answer" id="option3" value="option3">
       <label for="option3" id="option3">{{ option3 }}</label>
      </span>
      <span>
       <input type="radio" class="radio" name="answer" id="option4" value="option4">
       <label for="option4" id="option4">{{ option4 }}</label>
      </span>
    </div>

    <div class="next-button-container">
        <button onclick="prevQuestion()">Previous question</button>
        <button onclick="nextQuestion()" id="nextBtn">Next question</button>
    </div>
  </div>
    <script>
    function playSoundEffectFail() {
        const audio = new Audio("https://drive.google.com/uc?export=download&id=1cbO40W_UBsnnHdBH3oYjmnuomNzedriv");
        audio.play();
    }

    function playSoundEffectSuccess() {
        const audio = new Audio("https://drive.google.com/uc?export=download&id=1SoZJAMrhLMqQcsrLU1FJGVXpjBXveJbt");
        audio.play();

    }

        var correctAnswer = "{{ correct_Answer }}";
        function highlightOption(option) {
          var selectedAnswer = option.value;
          var selectedOp = document.getElementById(selectedAnswer.toString()).parentElement;
          var isCorrect = false;
          var score =document.getElementById("player-score").innerText;
          var question_number=document.getElementById("question_number").innerHTML;
          if (selectedAnswer == correctAnswer) {
            playSoundEffectSuccess();
            selectedOp.style.borderColor = '#03C03C';
            isCorrect = true;
            if("{{ question_num }}" == 10){
              google.colab.kernel.invokeFunction('handle_endGame');
            }
            else{

              google.colab.kernel.invokeFunction('handle_answer',[isCorrect, selectedAnswer], {});
            }

          } else {
            playSoundEffectFail();
            selectedOp.style.borderColor = 'red';
              document.getElementById("player-score").innerHTML= score-1;
             google.colab.kernel.invokeFunction('handle_answer',[isCorrect, selectedAnswer], {});
          }
        }
        var radioButtons = document.querySelectorAll('input[name="answer"]');
        radioButtons.forEach(function(radioButton) {
            radioButton.addEventListener('change', function() {
            highlightOption(this);
            });
        });
        function nextQuestion() {
            google.colab.kernel.invokeFunction('handle_nextQuestion');
        }
        function prevQuestion() {
            google.colab.kernel.invokeFunction('handle_prevQuestion');
        }
        function endGame() {
            google.colab.kernel.invokeFunction('handle_endGame');
        }

    </script>
</body>
</html>
'''
class Question:
  def __init__(self, question, option1, option2, option3, option4,correct_answer, is_correct=None, answer=None):

    self.question=question
    self.option1=option1
    self.option2=option2
    self.option3=option3
    self.option4=option4
    self.correct_answer=correct_answer
    self.is_correct=is_correct
    self.answer=answer
    self.mistakes=0

  def set_answer(self,is_correct,answer):
    self.is_correct=is_correct
    self.answer=answer

  def count_mistakes(self):
    self.mistakes += 1
    return self.mistakes


def loadQuestions():
    questions = []
    res = FBconn.get('/python_questions/', None)
    for question_key, question_data in res.items():
        q = Question(question_data["Question"],question_data["Option1"],question_data["Option2"], question_data["Option3"], question_data["Option4"], [question_data["Correct"].lower()])
        print(q.question,q.option1,q.correct_answer)
        questions.append(q)
    return questions

questions = loadQuestions()
questions[:10]

def displayQuestion(question_num, question, score):
    progress_percentage = (question_num / 10) * 100
    html_code = html_questions.replace('{{ score }}', str(score))
    html_code = html_code.replace('{{ question_num }}', str(question_num))
    html_code = html_code.replace('{{ question }}', question.question)
    html_code = html_code.replace('{{ option1 }}', question.option1)
    html_code = html_code.replace('{{ option2 }}', question.option2)
    html_code = html_code.replace('{{ option3 }}',  question.option3)
    html_code = html_code.replace('{{ option4 }}',  question.option4)
    html_code = html_code.replace('"{{ correct_Answer }}"', str(question.correct_answer))
    html_code = html_code.replace('{{ progress }}', str(progress_percentage))
    clear_output()
    display(HTML(html_code))

def displayNextQuestion():
    global numOfQuestion
    global score
    time.sleep(2)
    if(numOfQuestion < len(questions)-1):
        numOfQuestion += 1
        displayQuestion(numOfQuestion, questions[numOfQuestion], score)



def displayPrevQuestion():
    global numOfQuestion
    global score
    if(numOfQuestion > 0):
      numOfQuestion -=1
      displayQuestion(numOfQuestion, questions[numOfQuestion], score)



def handleAnswer(isCorrect,answer):
    global numOfQuestion
    global score
    questions[numOfQuestion].set_answer(isCorrect,answer)
    if isCorrect:
        score += 3
        if numOfQuestion != len(questions):
          displayNextQuestion()
        else:
          endGame()
    else:
      score -=1
      questions[numOfQuestion].count_mistakes()




def endGame():
  clear_output()
  run_end_game()
  #display(IPython.display.HTML(html_end_game))

google.colab.output.register_callback('handle_answer', handleAnswer)
google.colab.output.register_callback('handle_prevQuestion',displayPrevQuestion)
google.colab.output.register_callback('handle_nextQuestion', displayNextQuestion)
google.colab.output.register_callback('handle_endGame',endGame)

What is the keyword used to define a function in Python? def ['option1']
What is the result of 2 + 3 * 4? 20 ['option2']
What is the correct syntax for a for loop in Python? for i in range(10): ['option1']
Which of the following is a data type in Python? Integer ['option4']
Which of the following is NOT a Python built-in data type? List ['option']
Which Python framework is commonly used for web development? Pandas ['option3']
Which cloud computing platform is offered by IBM? AWS ['option4']
In Google Colab, True or False: You can execute shell commands directly within a code cell. True ['option1']
Which of the following data types in Python is used to store textual data? Integers ['option2']
Which company offers the AWS (Amazon Web Services) cloud platform? Microsoft ['option3']
Which cell type in Google Colab is used for creating rich text documentation? Code ['option4']
Which Python control structure is typically used for iterating over a sequence? for ['option1']
Which Google Cloud 

End game

In [52]:
tableCSS = "border-collapse: collapse; width: 100%; text-align: center; margin: 20px auto; border-radius: 10px; box-shadow: 0 0 20px rgba(0, 0, 0, 0.15); background-color: #2F2F2F;"
bodyCSS = "background-color: url(https://w0.peakpx.com/wallpaper/5/430/HD-wallpaper-video-game-eastward-sky-cloud.jpg);font-family: Arial, sans-serif;font-size: 1rem;line-height: 1.5;margin: 0;padding: 0;background-size: cover"
h1CSS = "color: #FFFFFF;font-size: 2.5rem;margin-bottom: 1em;text-align: center;font-weight: bold; letter-spacing: 3px;"
buttonCSS = "background-color: #4CAF50;border: none;color: white;padding: 15px 32px;text-align: center;text-decoration: none;display: block;font-size: 16px;margin: 20px auto;cursor: pointer;"
trCSS = "background-color: #4CAF50;color: black;padding: 10px 20px;text-align: center;"
tdCSS = "padding: 8px; border: 1px solid #ddd; color: #fff;"
tabCSS = "display: inline-block; background-color: #f1f1f1; border: none; outline: none; cursor: pointer; padding: 14px 16px; transition: 0.3s; font-size: 17px; display:flex;"
tab_containerCSS = "display: flex;justify-content: center; gap:5px;"
tabContentCSS = "display: none; padding: 6px 12px; border-top: none; animation: fadeEffect 1s;"

tabActiveCSS = "background-color: #ccc;"
fadeEffectCSS = "@keyframes fadeEffect {from {opacity: 0;}to {opacity: 1;}}"

container = "margin: 0 auto; max-width: 800px; padding: 20px;"
scoreCSS = "font-size: 24px; text-align: center;color: white; margin-bottom: 20px;"
scrollableDivCSS = "height: 300px; overflow: auto;"
thCSS = "padding: 8px; border: 1px solid #ddd; color: #fff; position: sticky; top: 0; background-color: #0db5b5; z-index: 1;"

In [19]:
tabFunctions = '''
    <script>
        function openTab(evt, tabName) {
  // Declare all variables
  var i, tabcontent, tablinks;

  // Get all elements with class="tabcontent" and hide them
  tabcontent = document.getElementsByClassName("tabcontent");
  for (i = 0; i < tabcontent.length; i++) {
    tabcontent[i].style.display = "none";
  }

  // Get all elements with class="tab" and remove the class "active"
  tablinks = document.getElementsByClassName("tab");
  for (i = 0; i < tablinks.length; i++) {
    tablinks[i].className = tablinks[i].className.replace(" active", "");
    tablinks[i].style.backgroundColor = "#f1f1f1"; // Reset the background color for all tab buttons
  }

  // Show the current tab, and add an "active" class to the button that opened the tab
  document.getElementById(tabName).style.display = "block";
  evt.currentTarget.className += " active";
  evt.currentTarget.style.backgroundColor = "#ccc"; // Set the background color for the active tab button
}

    </script>
'''

In [53]:
def run_end_game():
  try:
    temp = user.get_username()
  except:
    temp = None
  if temp is not None:
    # Import the IPython module
    import IPython
    from google.colab import output
    try:
      tempUsername = user.get_username()
    except:
      print("user did not loged in yet...")
      tempUsername = None
    # Define a function to get the index of a user by name
    def get_user_index(user_name):
        # Get the list of users from the Facebook API
        users = FBconn.get('/users/', None)

        # Iterate over the list of users and check if the name matches
        for i, user in enumerate(users.values()):
            if user['username'] == user_name:
                # Return the index of the user if a match is found
                return list(users.keys())[i]
        # Return None if no match is found
        return None

    # Get the list of users and their scores from the Facebook API
    users = FBconn.get('/users/', None)
    users = users.values()

    def get_score():
      global score
      return score


    def get_questions():
      global questions
      return questions
    questions=get_questions()
    # Create a dictionary of user names and scores using a dictionary comprehension
    list_of_scores = {user['name']: {user['total_score'], user["username"]} for user in users}

    # Sort the dictionary by score in descending order using the sorted() function and a lambda function as the key argument
    sorted_list_of_scores = [(user['username'], user['name'], user['total_score']) for user in reversed(sorted(users, key=lambda user: user['total_score']))]

    list_of_history = {}

    def get_user_data(tempUsername, FBconn):
        user_index = get_user_index(tempUsername)
        if user_index is None:
            print("DID NOT FIND THE USER")
            return None, None

        history = FBconn.get(f'/users/{user_index}/history/', None)
        if history is None:
            return None, None

        list_of_history = {item['question_id']: item['is_correct'] for item in history}

        return list_of_history

    if tempUsername != None:
      list_of_history = get_user_data(tempUsername, FBconn)


    # Create an HTML table for the leaderboard using f-strings and a list comprehension
    def generate_leaderboard_and_history_html(tempUsername, FBconn, sorted_list_of_scores, questions):
        table_html = f'''
        <div style="{scrollableDivCSS}">
        <table class="leaderboard" style="{tableCSS}">
            <tr>
                 <th style="{thCSS}">Rank</th>
                <th style="{thCSS}">Name</th>
                <th style="{thCSS}">Total Score</th>
            </tr>
            {''.join(f'<tr style="background-color: {"#5cabab" if user[0] == tempUsername else "transparent"}"><td>{"♕" if i == 0 else i+1}</td><td>{user[1]}</td><td>{user[2]}</td></tr>' for i, user in enumerate(sorted_list_of_scores))}
        </table>
        </div>
        '''

        table_html2 = f'''
        <div style="{scrollableDivCSS}">
        <table class="history" style="{tableCSS}">
            <tr>
                <th style="{thCSS}">Question Number</th>
                <th style="{thCSS}">Answered</th>
            </tr>
            {''.join(f'<tr><td>{q.question}</td><td>{"🗸" if q.mistakes==0 else f"you had {q.mistakes} mistakes"}</td></tr>' for q in questions)}
        </table>
        </div>
        '''

        return table_html, table_html2

    if(tempUsername != None):

      history_html,leaderboard_html = generate_leaderboard_and_history_html(tempUsername, FBconn, sorted_list_of_scores, questions)
    else:
      history_html = ''''''
      leaderboard_html = ''''''

    # Define a string containing the HTML, CSS, and JavaScript code to display the tables in tabs, add a background image to the body, and format the tables using CSS styles
    html_end_game = f'''
    <!DOCTYPE html>
    <html>
    <head>
        <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
        <title>End Game</title>
        <style>
            .tab {{
                {tabCSS}
            }}
            .tabcontent {{
                {tabContentCSS}
            }}
            .tab:hover {{
                {tabActiveCSS}
            }}
            .tab-container {{
                {tab_containerCSS}
            }}
            @keyframes fadeEffect {{
                {fadeEffectCSS}
            }}
            .container {{
                {container}
            }}
            .leaderboard {{
              <div style="height: 300px; overflow: auto;">
                {tableCSS}
              </div>
            }}
            .leaderboard th {{
                {trCSS}
            }}
            .leaderboard td {{
                {tdCSS}
            }}
            .history {{
              <div style="height: 300px; overflow: auto;">
                {tableCSS}
              </div>
            }}
            .history th {{
                {trCSS}
            }}
            .history td {{
                {tdCSS}
            }}

            .user-score-label{{
              {scoreCSS}
                animation: focus-in-contract-bck 1s cubic-bezier(0.250, 0.460, 0.450, 0.940) both;
            }}
            @keyframes focus-in-contract-bck {{
              0% {{
                letter-spacing: 1em;
                transform: translateZ(300px);
                filter: blur(12px);
                opacity: 0;
              }}
              100% {{
                transform: translateZ(12px);
                filter: blur(0);
                opacity: 1;
                }}
              }}

            .tracking-in-expand {{
                animation: tracking-in-expand 2s cubic-bezier(.215, .61, .355, 1.000) both;
            }}
            @keyframes tracking-in-expand {{
                0% {{
                    letter-spacing: -.5em;
                    opacity: 0;
                }}
                40% {{
                    opacity: .6;
                }}
                100% {{
                    opacity: 1;
                }}
            }}
        </style>
        <style>
            body {{
                background-image: url(https://w0.peakpx.com/wallpaper/76/395/HD-wallpaper-video-game-genshin-impact-cloud.jpg);
                background-size: cover;
            }}
            img {{
                width: 10px;
                height: 10px;
                float: left;
            }}
        </style>
        {tabFunctions}
        <script>


        function openQuestions(){{
          google.colab.kernel.invokeFunction('handle_quiz');
        }}

        function openMainScreen(){{

          google.colab.kernel.invokeFunction('handle_return_to_main_screen');
        }}

        function createGraph() {{
        const ctx = document.getElementById('myChart').getContext('2d');
        const myChart = new Chart(ctx, {{
          type: 'line',
          data: {{
            labels: ({[index + 1 for index, _ in enumerate(questions)]}),
            datasets: [{{
              label: 'Incorrect Answers',
              data: [], // Initialize with an empty array
              backgroundColor: [
                'rgba(30, 30, 30, 0.8)'
              ],
              borderColor: [
                'border:10px solid;border-image: radial-gradient(circle farthest-corner at center, rgba(30, 30, 30, 1) 10%, rgba(20, 20, 20, 1) 20%, rgba(10, 10, 10, 1) 35%, rgba(5, 5, 5, 1) 45%, rgba(0, 0, 0, 1) 55%, rgba(0, 0, 0, 1) 65%, rgba(0, 0, 0, 1) 80%, rgba(0, 0, 0, 1) 90%, rgba(0, 0, 0, 1) 100%) 10% round;'          ],
              borderWidth: 1
          }}]
          }},
          options: {{
            scales: {{
              y: {{
                beginAtZero: true
              }}
            }}
          }}
        }});

        // Calculate the cumulative sum of correct answers
        const mistakes = {[question.mistakes for question in questions]};
        const cumulativeMistakes = mistakes.map((mistake, index) => mistake);
        console.log({questions[0].mistakes})
        console.log(cumulativeMistakes)
        // Update the chart data and re-render the chart
        updateChartData(myChart, cumulativeMistakes);
      }}
        function updateChartData(chart, newData)
        {{chart.data.datasets[0].data = newData;
          chart.update();}}
        </script>
    </head>
    <body class="body" style="{bodyCSS}">
        <div class="container">
            <h1 class="tracking-in-expand" style="{h1CSS}">End Game 🦄</h1>
            <script>
            console.log({score})
            function getUserScore(){{
              google.colab.kernel.invokeFunction('get_score')
            }}
            </script>
            <div class="user-score-label">Your Score: {score}☁</div>
            <div class="tab-container">
                <button class="tab" onclick="openTab(event, 'Table1')">Game review</button>
                <button class="tab" onclick="openTab(event, 'Table2')">Leaderboard</button>
                <!-- Add a new tab button for the graph -->
                <button class="tab" onclick="openTab(event, 'Graph'); createGraph();">Graph</button>
            </div>
            <div id="Table1" class="tabcontent" style="display:block;">
                {leaderboard_html}
            </div>
            <div id="Table2" class="tabcontent">
                {history_html}
            </div>

              <!-- Graph -->
              <!-- Add a new tab content div for the graph -->
              <div id="Graph" class="tabcontent">
                <canvas id="myChart"></canvas>
              </div>

            <button class="play-again-button" style="{buttonCSS}" onmouseover="this.style.backgroundColor='#3e8e41'" onmouseout="this.style.backgroundColor='#4CAF50'" onclick="openQuestions()">Play Again</button>
            <button class="return-home-button" style="{buttonCSS}; background-color: red;" onmouseover="this.style.backgroundColor='#FF4500'" onmouseout="this.style.backgroundColor='red'" onclick="openMainScreen()">Return Home</button>
        </div>
        <script src="https://canvasjs.com/assets/script/canvasjs.min.js"></script>
    </body>
    </html>
    '''


    def mainScreen():
      clear_output()
      display(IPython.display.HTML(html_mainScreen))


    google.colab.output.register_callback('handle_return_to_main_screen', mainScreen)

    google.colab.output.register_callback('handle_quiz', openQuiz)


    #%store html_end_game
    display(IPython.display.HTML(html_end_game))



Manager screen

In [8]:
from IPython.display import display, HTML, clear_output
import google.colab.output
from firebase import firebase
import json

FBconn = firebase.FirebaseApplication('https://pythongame-a4fc4-default-rtdb.firebaseio.com/', None)
html_content = '''
<head>
  <link rel="stylesheet" type="text/css" href="https://drive.google.com/uc?id=1WQCIPgq9rt5VkLC2H89zB_-esW7iaLGT">
</head>
<body>
  <h1>Welcome to the Home Page</h1>
  <div id="bottonContainer">
  <button class="button" onclick="editQuestion()">Edit Question</button>
  <button class="button" onclick="openForm()">Insert New Question</button>
  </div>

  <div class="editQuestionContainer">
    <div id="formContainer" style="display: none;">
      <button class="close-btn" type="button"  onclick="closeForm()">X</button>
      <h2 id="formTitle">Insert New Question</h2>
      <form id="questionForm">
        <label for="questionName">Question Name:</label>
        <textarea id="questionName" name="content" placeholder="Write the question.." style="height:100px" required></textarea>
        <label for="ans1">Answer 1:</label>
        <input type="text" id="ans1" name="ans1" required><br><br>
        <label for="ans2">Answer 2:</label>
        <input type="text" id="ans2" name="ans2" required><br><br>
        <label for="ans3">Answer 3:</label>
        <input type="text" id="ans3" name="ans3" required><br><br>
        <label for="ans4">Answer 4:</label>
        <input type="text" id="ans4" name="ans4" required><br><br>
        <label for="correctAns">Correct answer:</label>
        <select id="correctAns" name="correctAns" class="classic" required>
          <option value="">Select the correct answer</option>
          <option value="Option1">Answer 1</option>
          <option value="Option2">Answer 2</option>
          <option value="Option3">Answer 3</option>
          <option value="Option4">Answer 4</option>

        </select>
        <input type="text" id="idq" name="id" style="display: none;"><br><br>
        <button class="button" type="button" id="submit" onclick="saveQuestion()">Save</button>
        <button class="button" type="button" style="display: none; background: red" id="delete" onclick="deleteQuestion()">Delete Question</button>
      </form>
    </div>
    <div class="select-question" id="questionListContainer" style="display: none; text-align: center;">
      <h2>Select a Question to Edit:</h2>
      <div class="selectContainer" style="display: flex; justify-content: center;">
        <select id="questionList" onchange="selectQuestion()" class="classic">
          <option value="">Select a question</option>
        </select>
  </div>
</div>
  </div>


</body>
'''
javascript_code = '''
<script>
function editQuestion() {
  var deleteBtn = document.getElementById("delete");
  deleteBtn.style.display = "block"; // Display the delete button when a question is selected
  var bottonContainer = document.getElementById("bottonContainer");
  bottonContainer.style.display = "none";
  var result=google.colab.kernel.invokeFunction('get_questions_size', [], {});
  result.then(function(value) {
      var n = parseInt(value.data['text/plain']);
      createQuestionCB(n);
      var formContainer = document.getElementById("formContainer");
      var questionListContainer = document.getElementById("questionListContainer");
      formContainer.style.display = "none";
      questionListContainer.style.display = "block";
  });

}
function createQuestionCB(n){
  var cmb = document.getElementById("questionList");
  for (var i = 1; i<=n; i++){
    var opt = document.createElement('option');
    opt.value = i;
    opt.innerHTML = "Question"+i;
    cmb.appendChild(opt);
}
}
function selectQuestion() {
  var questionList = document.getElementById("questionList");
  var selectedIndex = questionList.value;
  var formContainer = document.getElementById("formContainer");
  var formTitle = document.getElementById("formTitle");
  var questionForm = document.getElementById("questionForm");
  var deleteBtn = document.getElementById("delete");
  if (selectedIndex!== "") {
    var result=google.colab.kernel.invokeFunction('get_firebase_data', [selectedIndex], {});
    result.then(function(value) {
      var data = value.data['text/plain']
    // Remove the parentheses from the string

      // Remove the parentheses and single quotes from the string
    data = data.slice(2, -2);

   // Split the string at the closing curly brace
    var dataArray = data.split('}');

   // Trim whitespace from the values and add back the closing curly brace
   dataArray[0] = (dataArray[0].trim() + '}');
   dataArray[1] = dataArray[1].trim();
     var key=dataArray[1].substring(5);
      data= dataArray[0]
      data=JSON.parse(data)
      openForm();
      var questionData = data;
      questionForm.elements["questionName"].value = questionData["Question"];
      questionForm.elements["ans1"].value = questionData["Option1"];
      questionForm.elements["ans2"].value = questionData["Option2"];
      questionForm.elements["ans3"].value = questionData["Option3"];
      questionForm.elements["ans4"].value = questionData["Option4"];
      questionForm.elements["correctAns"].value = questionData["Correct"];
      questionForm.elements["id"].value =key;
      var formTitle = document.getElementById("formTitle");
      formTitle.innerHTML = "Edit Question";
      deleteBtn.style.display = "block"; // Display the delete button when a question is selected

  });
  } else {
    formContainer.style.display = "none";
  }
}

function openForm() {
  cleanForm()
  var bottonContainer = document.getElementById("bottonContainer");
  bottonContainer.style.display = "none";
  var formContainer = document.getElementById("formContainer");
  var questionListContainer = document.getElementById("questionListContainer");
  formContainer.style.display = "block";
  questionListContainer.style.display = "none";
  var formTitle = document.getElementById("formTitle");
}

function closeForm(){
  cleanForm();
  formContainer.style.display = "none";
  var bottonContainer = document.getElementById("bottonContainer");
  bottonContainer.style.display = "block";
}

function saveQuestion(){
  var questionForm = document.getElementById("questionForm");
  var question = [questionForm.elements["correctAns"].value,
  questionForm.elements["ans1"].value,
  questionForm.elements["ans2"].value,
  questionForm.elements["ans3"].value,
  questionForm.elements["ans4"].value,
  questionForm.elements["questionName"].value,questionForm.elements["id"].value];
  console.log(question)
  google.colab.kernel.invokeFunction('handleAddQuestion', [question], {});
  closeForm()

}

function deleteQuestion() {
  var questionForm = document.getElementById("questionForm");
  var questionId = questionForm.elements["id"].value;
  if(questionId !== "") {
    google.colab.kernel.invokeFunction('handleDeleteQuestion', [questionId], {});
    closeForm()
    console.log("Question deleted");
  }
}

function cleanForm(){
  formTitle.innerHTML = "Insert New Question";
  questionForm.elements["correctAns"].value=""
  questionForm.elements["questionName"].value = "";
  questionForm.elements["ans1"].value = "";
  questionForm.elements["ans2"].value = "";
  questionForm.elements["ans3"].value = "";
  questionForm.elements["ans4"].value = "";
  questionForm.elements["id"].value = "";
  var deleteBtn = document.getElementById("delete");
  deleteBtn.style.display = "none"; // Display the delete button when a question is selected
  var questionCmb = document.getElementById("questionList");
  questionCmb.innerHTML=""; // clear cmb
}
</script>
'''
def get_firebase_data(selected_index):
  index=int(selected_index[0])
  fB=FBconn.get('/python_questions/',None)
  for i, key in enumerate(fB):
    if i==index:
      question_data = fB[key]
      json_data = json.dumps(question_data)  # Convert to JSON string using double quotes
      return json_data,key
  return {}

def get_questions_size():
  fB=FBconn.get('/python_questions/',None)
  return len(fB)

def insert_new_data(question):
  data={
      'Correct':question[0],
      'Option1':question[1],
      'Option2':question[2],
      'Option3':question[3],
      'Option4':question[4],
      'Question':question[5]
  }
  if(question[6]!=''):
   FBconn.put('/python_questions', question[6], data)
  else:
   result=FBconn.post('/python_questions/',data)

def handleDeleteQuestion(question_id):
  FBconn.delete('/python_questions', question_id)

google.colab.output.register_callback('handleDeleteQuestion', handleDeleteQuestion)


google.colab.output.register_callback('get_firebase_data', get_firebase_data)
google.colab.output.register_callback('handleAddQuestion', insert_new_data)
google.colab.output.register_callback('get_questions_size', get_questions_size)
manager_code=html_content + javascript_code
%store manager_code
display(HTML(manager_code))

Stored 'manager_code' (str)
